In [19]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

base_string = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   

def make_sweep_str(pfile, pars, group):
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        base_str += ' ' + k + f'={pars[k]}'
        name += k.split('.')[-1]+f':{pars[k]}x'
        name = name.replace('[', '')
        name = name.replace(']', '')
        name = name.replace(',', 'x')
    name = name[:-1]
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [4]:
group = 'sweep_m3'
pfile = 'Dlg1_m3'

variable_col = param_iter()
variable_col.add('noise.theta', 29.41*1.5)
variable_col.add('bg_estimation.fractal.scale', 2000, 3000)
variable_col.add('bg_estimation.fractal.persistence', 0.5,0.75)
variable_col.add('prob_generator.high', 0.0003, 0.00015)
par_prod = variable_col.param_product()

In [5]:
group = 'sweep_bg_1a'
pfile = 'MERFISH_starfish_ch0_f5'

variable_col = param_iter()
variable_col.add('bg_estimation.fractal.scale', 0)
variable_col.add('bg_estimation.smoothing.smoothing_filter_size', 7,9)
par_prod = variable_col.param_product()

In [83]:
# group = 'sweep_ctx_4'
# pfile = 'MERFISH_starfish_c2'

# variable_col = param_iter()
# variable_col.add('microscope.norm','none', 'max')
# par_prod = variable_col.param_product()

In [159]:
group = 'sweep_ae_4'
pfile = 'MERFISH_ci_2d_ae4'

variable_col = param_iter()

variable_col.add('training.int.enabled',True, False)
par_prod = variable_col.param_product()

In [160]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_ci_2d_ae4 training.int.enabled=True run_name=enabled:True output.group=sweep_ae_4

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_ci_2d_ae4 training.int.enabled=False run_name=enabled:False output.group=sweep_ae_4



In [127]:
f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/{sweep}/*'

'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_ae_1/*'

In [156]:
sweep = 'sweep_ae_2'

dirs = glob.glob(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/{sweep}/*')
with open('run_slurm.sh', 'w') as f:
    for d in dirs:
        name = d.split('/')[-1]
        exec_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/merfish_eval.py run_name={name} sweep_name={sweep}'  + '\n'
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/merfish_eval.py run_name=shuffle_ch:Falsexenabled:Truexseed:3 sweep_name=sweep_ae_2

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/merfish_eval.py run_name=shuffle_ch:Falsexenabled:Truexseed:0 sweep_name=sweep_ae_2

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/merfish_eval.py run_name=shuffle_ch:Truexenabled:Truexseed:2 sweep_name=sweep_ae_2

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/hom

In [120]:
group = 'sep_models_ch0_3'
pfile = 'CL_dlg1_ch0_f1'

variable_col = param_iter()
variable_col.add('data_path.image_path', 
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p1*.tiff',
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p2*.tiff',
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p3*.tiff')

par_prod = variable_col.param_product()

In [40]:
group = 'sweep_fig9'
pfile = 'fig_sim_m9'

variable_col = param_iter()
variable_col.add('training.mic.enabled', True, False)
variable_col.add('training.int.enabled', True, False)

par_prod = variable_col.param_product()

In [93]:
group = 'sweep_psf2'
pfile = 'N2_352_f1'

variable_col = param_iter()
variable_col.add('microscope.psf_noise', 0.1, 0.2)
variable_col.add('microscope.double_random', True, False)
variable_col.add('model.f_maps', 32, 42)

par_prod = variable_col.param_product()

In [36]:
from decode_fish.imports import *

psfn_3: psf_noise = 0.5
psfn_4: psf_noise = 0.75
psfn_5: psf_noise = 1.
psfn_6: psf_noise = 1. double random
psfn_7: psf_noise = 2. double random
psfn_8: psf_noise = 3. double random

In [88]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1', 'CL_dlg1_ch0', 'CL_dlg1_ch1']
exps = ['CL_dlg1_ch1']
ver = 'f1'
name = 'f1_psfn_5'

with open('run_slurm.sh', 'w') as f:
    for exp in exps:  
        cfg = OmegaConf.load(f'config/experiment/{exp}_{ver}.yaml')
#         exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name}' + '\n'
        exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name} noise.theta={cfg.noise.theta*1.5} microscope.psf_noise=0.1' + '\n'
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=CL_dlg1_ch1_f1  run_name=CL_dlg1_ch1_f1_psfn_5 noise.theta=75.0 microscope.psf_noise=0.1



In [ ]:
# bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/sim_eval.py sweep_name=fig7

In [ ]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1']
# exps = [ 'scrib', 'Dlg1']
ver = 'f1'
name = 'f1_none'